# Initial Setup

In [1]:
import os
import sys
from typing import List,Optional, Dict, Any, Tuple
import pandas as pd
import numpy as np

# llama index imports
import llama_index.core
from llama_index.core import SimpleDirectoryReader,VectorStoreIndex,SummaryIndex, StorageContext, Settings, load_index_from_storage, Response
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.node_parser import SentenceSplitter, CodeSplitter, LangchainNodeParser
from llama_index.core.tools import FunctionTool,QueryEngineTool
from llama_index.core.vector_stores import MetadataFilters,FilterCondition
from llama_index.core.objects import ObjectIndex
from llama_index.readers.file import IPYNBReader, PandasCSVReader
from llama_index.experimental.query_engine import PandasQueryEngine
from llama_index.experimental.query_engine.pandas import (
    PandasInstructionParser,
)
from llama_index.core.prompts import PromptTemplate
from llama_index.core.query_pipeline import (
    QueryPipeline as QP,
    Link,
    InputComponent,
    AgentFnComponent,
    AgentInputComponent
)
from llama_index.readers.file import IPYNBReader, PandasCSVReader
from llama_index.experimental.query_engine import PandasQueryEngine
from llama_index.experimental.query_engine.pandas import (
    PandasInstructionParser,
)
from llama_index.core.prompts import PromptTemplate
from llama_index.core.query_pipeline import (
    QueryPipeline as QP,
    Link,
    InputComponent,
    CustomQueryComponent,
    FnComponent
)
from llama_index.core.callbacks import CallbackManager

# llama index agent imports
from llama_index.core.agent import FunctionCallingAgentWorker, ReActAgent, Task, AgentChatResponse, AgentRunner, QueryPipelineAgentWorker

# llama index llms and embeddings imports
from llama_index.llms.mistralai import MistralAI
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.mistralai import MistralAIEmbedding
from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# custom package imports
from llama_index.packs.tables.chain_of_table.base import ChainOfTableQueryEngine, serialize_table

# langchain imports
from langchain.text_splitter import RecursiveCharacterTextSplitter, Language

# tools
import nest_asyncio # to allow running async functions in jupyter
import chromadb # persistent storage for vectors
# import nbconvert
import tree_sitter
import tree_sitter_languages
import phoenix as px
from pyvis.network import Network

d:\conda_envs\rag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Global Variables

In [7]:
temperture = 0.0 #for deterministic results

llm_model = "mistral-large-latest"
MISTRAL_API_KEY =  "BWdlihu9sUh5P2g3bHnzjAaHiT4anTVH"
os.environ["MISTRAL_API_KEY"] = MISTRAL_API_KEY
llm = MistralAI(model=llm_model, temperature=temperture)

# llm_model = "codellama"
# llm = Ollama(model=llm_model, request_timeout=1200.0, base_url="http://localhost:11434", temperature=temperture)

nest_asyncio.apply() # to allow running async functions in jupyter
px.launch_app()
llama_index.core.set_global_handler("arize_phoenix")
net = Network(notebook=True, cdn_resources="in_line", directed=True)

WARNI [phoenix.session.session] Existing running Phoenix instance detected! Shutting it down and starting a new instance...


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


# Implementation

## Load Data



In [3]:
# loading file
file_path = "./data_csv/Sepsis_Processed_IC.csv"
df = pd.read_csv(file_path)

## Query Pipeline Setup

In [8]:
# create prompt modules
instruction_str = (
    "1. Convert the query to executable Python code using Pandas.\n"
    "2. The final line of code should be a Python expression that can be called with the `eval()` function.\n"
    "3. The code should represent a solution to the query.\n"
    "4. PRINT ONLY THE EXPRESSION.\n"
    "5. Do not quote the expression.\n"
)

pandas_prompt_str = (
    "You are working with a pandas dataframe in Python.\n"
    "The name of the dataframe is `df`.\n"
    "This is the result of `print(df.head())`:\n"
    "{df_str}\n\n"
    "This is a brief description of the dataframe:\n"
    "This table records patient data related to sepsis diagnosis. The Target column indicates whether the patient had sepsis. The duration_since_reg column describes the patient's stay after admission in days. Diagnosis-related columns detail specific diagnostic results and associated codes. The dataset includes patient demographics age, clinical measurements (crp, lacticacid, leucocytes), and diagnostic procedures (diagnosticartastrup, diagnosticblood, etc.). It also tracks clinical criteria for sepsis (sirscritheartrate, sirscritleucos, etc.), resource usage, and event transitions (e.g., CRP => ER Triage). Additional columns capture organ dysfunction, hypotension, hypoxia, suspected infection, and treatment details like infusions and oliguria. The dataset covers the transitions between various clinical events, highlighting the pathways in the patient's diagnostic and treatment journey. ER here refers to the emergency room.\n\n"
    "Follow these instructions:\n"
    "{instruction_str}\n"
    "Query: {query_str}\n\n"
    "Expression:"
)
response_synthesis_prompt_str = (
    "Given an input question, synthesize a response from the query results.\n"
    "Query: {query_str}\n\n"
    "Pandas Instructions (optional):\n{pandas_instructions}\n\n"
    "Pandas Output: {pandas_output}\n\n"
    "Response: "
)

pandas_prompt = PromptTemplate(pandas_prompt_str).partial_format(
    instruction_str=instruction_str, df_str=df.head(5)
)
pandas_output_parser = PandasInstructionParser(df)
response_synthesis_prompt = PromptTemplate(response_synthesis_prompt_str)

In [9]:
# define query pipeline with modules
qp_table = QP(
    modules={
        "input": InputComponent(),
        "pandas_prompt": pandas_prompt,
        "llm1": llm,
        "pandas_output_parser": pandas_output_parser,
        "response_synthesis_prompt": response_synthesis_prompt,
        "llm2": llm,
    },
    verbose=True,
)

qp_table.add_chain(["input", "pandas_prompt", "llm1", "pandas_output_parser"])
qp_table.add_links(
    [
        Link("input", "response_synthesis_prompt", dest_key="query_str"),
        Link(
            "llm1", "response_synthesis_prompt", dest_key="pandas_instructions"
        ),
        Link(
            "pandas_output_parser",
            "response_synthesis_prompt",
            dest_key="pandas_output",
        ),
    ]
)
# add link from response synthesis prompt to llm2
qp_table.add_link("response_synthesis_prompt", "llm2")


In [6]:
# create diagram of the query pipeline
net.from_nx(qp_table.clean_dag)
net.show("qp_table.html")

qp_table.html


## Query Pipeline run

In [10]:
# run query
response = qp_table.run(
    query_str="how many positive cases?",
)

> Running module input with input: 
query_str: how many positive cases?

> Running module pandas_prompt with input: 
query_str: how many positive cases?

> Running module llm1 with input: 
messages: You are working with a pandas dataframe in Python.
The name of the dataframe is `df`.
This is the result of `print(df.head())`:
  case_id  Target  age  max_activity_count  duration_since_reg  crp  \
0...

> Running module pandas_output_parser with input: 
input: assistant: df[df['Target'] == 1].shape[0]

> Running module response_synthesis_prompt with input: 
query_str: how many positive cases?
pandas_instructions: assistant: df[df['Target'] == 1].shape[0]
pandas_output: 98

> Running module llm2 with input: 
messages: Given an input question, synthesize a response from the query results.
Query: how many positive cases?

Pandas Instructions (optional):
df[df['Target'] == 1].shape[0]

Pandas Output: 98

Response: 



In [11]:
# query result
print(response.message.content)

Based on the data provided, there are 98 positive cases.


In [12]:
# run query
response = qp_table.run(
    query_str="what is the average time patients spend in the hospital?",
)
# query result
print("final response: ================================================================================================================================================================================================")
print(response.message.content)

> Running module input with input: 
query_str: what is the average time patients spend in the hospital?

> Running module pandas_prompt with input: 
query_str: what is the average time patients spend in the hospital?

> Running module llm1 with input: 
messages: You are working with a pandas dataframe in Python.
The name of the dataframe is `df`.
This is the result of `print(df.head())`:
  case_id  Target  age  max_activity_count  duration_since_reg  crp  \
0...

> Running module pandas_output_parser with input: 
input: assistant: df['duration_since_reg'].mean()

> Running module response_synthesis_prompt with input: 
query_str: what is the average time patients spend in the hospital?
pandas_instructions: assistant: df['duration_since_reg'].mean()
pandas_output: 11.568710217755443

> Running module llm2 with input: 
messages: Given an input question, synthesize a response from the query results.
Query: what is the average time patients spend in the hospital?

Pandas Instructions (optio

## Model Evaluation Agent Setup

In [13]:
# evaluate model
def evaluate_model(model_name:str) -> dict:
    """Load the trained model, evaluation data and evaluate the loaded model."""
    from sklearn.metrics import roc_auc_score, roc_auc_score, average_precision_score, confusion_matrix, f1_score, matthews_corrcoef
    from joblib import dump, load
    from xgboost import XGBClassifier

    model_save_path = 'models'
    model = load(f'./{model_save_path}/{model_name}.joblib')
    X_test = pd.read_csv('./data_python/Sepsis_X_test.csv')
    y_test = pd.read_csv('./data_python/Sepsis_y_test.csv')

    pred_prob = model.predict_proba(X_test) # get the prediction probabilities for the test set
    predictions = model.predict(X_test) # get the predictions for the test set

    roc_auc = roc_auc_score(y_test, pred_prob[:,1]) # calculate the roc auc score
    average_precision = average_precision_score(y_test, pred_prob[:,1]) # calculate the
    mcc =  matthews_corrcoef(y_test, predictions)
    f1_macro = f1_score(y_test, predictions, average='macro')
    cm = confusion_matrix(y_test, predictions)

    return {"roc_auc":roc_auc, "average_precision":average_precision, "mcc":mcc, "f1_macro":f1_macro, "confusion_matrix":cm}

# create tools
evaluate_model_tool = FunctionTool.from_defaults(name="evaluate_model", fn=evaluate_model)
tools = [evaluate_model_tool]

# setup ReAct agent
# model_agent_prompt = """You are a proficient python developer. Respond with the syntactically correct code for the question below. Make sure you follow these rules:
#                                         1. Use context to understand the APIs and how to use them.
#                                         2. Ensure all the requirements in the question are met.
#                                         3. Ensure the output code syntax is correct.
#                                         4. All required dependencies should be imported above the code.
#                                         Question:
#                                         {question}
#                                         Context:
#                                         {context}
#                                         Helpful Response:"""
# model_agent_prompt = PromptTemplate(model_agent_prompt)
agent_model = ReActAgent.from_tools(tools=tools, 
                                     llm=llm,
                                     verbose=True)
# agent_model.update_prompts({"agent_worker:system_prompt": model_agent_prompt})

ERROR [asyncio] Task was destroyed but it is pending!
task: <Task pending name='Task-5' coro=<BulkInserter._bulk_insert() running at d:\conda_envs\rag\lib\site-packages\phoenix\db\bulk_inserter.py:103> wait_for=<Future pending cb=[Task.__wakeup()]>>


## Model Evaluation Agent run

In [14]:
# response = await agent.achat("what is the auc_roc score of the trained model?")
response = agent_model.query("what is the auc_roc score of the XGBoost model?")
print(str(response))

Thought: The current language of the user is: English. I need to use the 'evaluate_model' tool to help me answer the question.
Action: evaluate_model
Action Input: {'model_name': 'XGBoost'}


d:\conda_envs\rag\lib\site-packages\xgboost\core.py:158: UserWarning: [19:06:26] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\data\../common/error_msg.h:80: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  warnings.warn(smsg, UserWarning)


Observation: {'roc_auc': 0.8659217877094972, 'average_precision': 0.7481294952146433, 'mcc': 0.7002038117109098, 'f1_macro': 0.8472989564149784, 'confusion_matrix': array([[176,   3],
       [  7,  13]], dtype=int64)}
Thought: The current language of the user is: English. I have the information needed to answer the question without using any more tools.
Answer: The AUC-ROC score of the XGBoost model is 0.8659217877094972.
The AUC-ROC score of the XGBoost model is 0.8659217877094972.


## Multi Agent Setup

In [15]:
# methods for running the agents / query pipelines
def run_agent(query: str) -> str:
    """Run the agent model on the query to get evaluation results from trained model."""
    response = agent_model.query(query)
    return str(response)

def run_query_pipeline(query: str) -> str:
    """Run the query pipeline to analyze dataset for the given query."""
    response = qp_table.run(
        query_str=query,
    )
    return str(response.message.content)

# create tools
run_agent_tool = FunctionTool.from_defaults(name="run_agent", fn=run_agent)
run_query_pipeline_tool = FunctionTool.from_defaults(name="run_query_pipeline", fn=run_query_pipeline)
agent_tools = [run_agent_tool, run_query_pipeline_tool]

top_level_agent_prompt = """
                You are designed to help with a variety of tasks, from answering questions \
                to providing summaries to other types of analyses.

                ## Tools
                You have access to a wide variety of tools. You are responsible for using
                the tools in any sequence you deem appropriate to complete the task at hand.
                This may require breaking the task into subtasks and using different tools
                to complete each subtask.

                You have access to the following tools:
                {tool_desc}

                ## Output Format
                To answer the question, please use the following format.

                ```
                Thought: I need to use a tool to help me answer the question.
                Action: tool name (one of {tool_names}) if using a tool.
                Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
                ```

                Please ALWAYS start with a Thought.

                Please use a valid JSON format for the Action Input. Do NOT do this {{'input': 'hello world', 'num_beams': 5}}.

                If this format is used, the user will respond in the following format:

                ```
                Observation: tool response
                ```

                You should keep repeating the above format until you have enough information
                to answer the question without using any more tools. At that point, you MUST respond
                in the one of the following two formats:

                ```
                Thought: I can answer without using any more tools.
                Answer: [your answer here]
                ```

                ```
                Thought: I cannot answer the question with the provided tools.
                Answer: Sorry, I cannot answer your query.
                ```

                ## Additional Rules
                - You MUST obey the function signature of each tool. Do NOT pass in no arguments if the function expects arguments.
                - For queries that clearly involve data retrieval or manipulation (like 'analyze sales data', 'show trends in data'), use 'run_query_pipeline'.
                - For queries that directly relate to model performance or evaluation (like 'what is the AUC_ROC score', 'evaluate the prediction accuracy'), use 'run_agent'.

                ## Current Conversation
                Below is the current conversation consisting of interleaving human and assistant messages.
                """
top_level_agent_prompt = PromptTemplate(top_level_agent_prompt)
agent = ReActAgent.from_tools(tools=agent_tools, 
                                    llm=llm, 
                                    verbose=True)
agent.update_prompts({"agent_worker:system_prompt": top_level_agent_prompt})

In [16]:
# response = await agent.achat("what is the auc_roc score of the trained model?")
# response = agent.query("what is the auc_roc score of the trained XGBoost model?")
response = agent.query("how many positive cases?")
print(str(response))

Thought: The user is asking for a specific count of positive cases, which involves data retrieval. I should use the 'run_query_pipeline' tool to answer this question.
Action: run_query_pipeline
Action Input: {'query': 'how many positive cases'}
> Running module input with input: 
query_str: how many positive cases

> Running module pandas_prompt with input: 
query_str: how many positive cases

> Running module llm1 with input: 
messages: You are working with a pandas dataframe in Python.
The name of the dataframe is `df`.
This is the result of `print(df.head())`:
  case_id  Target  age  max_activity_count  duration_since_reg  crp  \
0...

> Running module pandas_output_parser with input: 
input: assistant: df['Target'].sum()

> Running module response_synthesis_prompt with input: 
query_str: how many positive cases
pandas_instructions: assistant: df['Target'].sum()
pandas_output: 98

> Running module llm2 with input: 
messages: Given an input question, synthesize a response from the qu

In [14]:
prompt_dict = agent.get_prompts()
for k, v in prompt_dict.items():
    print(f"Prompt: {k}\n\nValue: {v.template}")

Prompt: agent_worker:system_prompt

Value: 
                You are designed to help with a variety of tasks, from answering questions                 to providing summaries to other types of analyses.

                ## Tools
                You have access to a wide variety of tools. You are responsible for using
                the tools in any sequence you deem appropriate to complete the task at hand.
                This may require breaking the task into subtasks and using different tools
                to complete each subtask.

                You have access to the following tools:
                {tool_desc}

                ## Output Format
                To answer the question, please use the following format.

                ```
                Thought: I need to use a tool to help me answer the question.
                Action: tool name (one of {tool_names}) if using a tool.
                Action Input: the input to the tool, in a JSON format representing the kwarg

## Multi Agent run

In [17]:
# response = await agent.achat("what is the auc_roc score of the trained model?")
response = agent.query("how many positive cases?")
print(f'===========================final response============================\n{str(response)}')

Thought: The user is asking for a count of positive cases, which involves data retrieval. I should use the 'run_query_pipeline' tool to answer this question.
Action: run_query_pipeline
Action Input: {'query': 'count positive cases'}
> Running module input with input: 
query_str: count positive cases

> Running module pandas_prompt with input: 
query_str: count positive cases

> Running module llm1 with input: 
messages: You are working with a pandas dataframe in Python.
The name of the dataframe is `df`.
This is the result of `print(df.head())`:
  case_id  Target  age  max_activity_count  duration_since_reg  crp  \
0...

> Running module pandas_output_parser with input: 
input: assistant: df['Target'].sum()

> Running module response_synthesis_prompt with input: 
query_str: count positive cases
pandas_instructions: assistant: df['Target'].sum()
pandas_output: 98

> Running module llm2 with input: 
messages: Given an input question, synthesize a response from the query results.
Query: c

In [18]:
# response = await agent.achat("what is the auc_roc score of the trained model?")
response = agent.query("what is the auc_roc score of the trained XGBoost model?")
print(f'===========================final response============================\n{str(response)}')

Thought: This question involves model performance evaluation, specifically asking for the AUC_ROC score of the trained XGBoost model. I should use the 'run_agent' tool to answer this.
Action: run_agent
Action Input: {'query': 'auc_roc_score_xgboost'}
Thought: The current language of the user is: English. The user is asking about the AUC ROC score for the XGBoost model. I need to use the 'evaluate_model' tool to help me answer this question.
Action: evaluate_model
Action Input: {'model_name': 'xgboost'}
Observation: {'roc_auc': 0.8659217877094972, 'average_precision': 0.7481294952146433, 'mcc': 0.7002038117109098, 'f1_macro': 0.8472989564149784, 'confusion_matrix': array([[176,   3],
       [  7,  13]], dtype=int64)}
Thought: The current language of the user is: English. I have the results of the XGBoost model evaluation. The AUC ROC score is 0.8659217877094972.
Answer: The AUC ROC score for the XGBoost model is 0.8659217877094972.
Observation: The AUC ROC score for the XGBoost model is

In [19]:
# response = await agent.achat("what is the auc_roc score of the trained model?")
response = agent.query("what is the average time patients spend in the hospital?")
print(f'===========================final response============================\n{str(response)}')

Thought: This question involves data retrieval and manipulation, specifically calculating the average time patients spend in the hospital. I should use the 'run_query_pipeline' tool to analyze the dataset and find this information.
Action: run_query_pipeline
Action Input: {'query': 'average time patients spend in the hospital'}
> Running module input with input: 
query_str: average time patients spend in the hospital

> Running module pandas_prompt with input: 
query_str: average time patients spend in the hospital

> Running module llm1 with input: 
messages: You are working with a pandas dataframe in Python.
The name of the dataframe is `df`.
This is the result of `print(df.head())`:
  case_id  Target  age  max_activity_count  duration_since_reg  crp  \
0...

> Running module pandas_output_parser with input: 
input: assistant: df['duration_since_reg'].mean()

> Running module response_synthesis_prompt with input: 
query_str: average time patients spend in the hospital
pandas_instruct